In [19]:
import csv
import random
import math
import pandas as pd

In [25]:
d=pd.read_csv("5-dataset.csv")

In [26]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766 entries, 0 to 765
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   6       766 non-null    int64  
 1   148     766 non-null    int64  
 2   72      766 non-null    int64  
 3   35      766 non-null    int64  
 4   0       766 non-null    int64  
 5   33.6    766 non-null    float64
 6   0.627   766 non-null    float64
 7   50      766 non-null    int64  
 8   1       766 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.0 KB


In [27]:
d.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [28]:
def loadcsv(filename):
    lines = csv.reader(open(filename, "r"));
    dataset = list(lines)
    for i in range(len(dataset)):
       dataset[i] = [float(x) for x in dataset[i]]
         
    return dataset

In [29]:
def splitdataset(dataset, splitratio):
    trainsize = int(len(dataset) * splitratio);
    trainset = []
    copy = list(dataset);    
    while len(trainset) < trainsize:
        index = random.randrange(len(copy));       
        trainset.append(copy.pop(index)) 
    return [trainset, copy]

In [30]:
def separatebyclass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [31]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [32]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [33]:
def summarize(dataset): #creates a dictionary of classes
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)];
    del summaries[-1] #excluding labels +ve or -ve
    return summaries

In [34]:
def summarizebyclass(dataset):
    separated = separatebyclass(dataset); 
    #print(separated)
    summaries = {}
    for classvalue, instances in separated.items(): 
#for key,value in dic.items()
#summaries is a dic of tuples(mean,std) for each class value        
        summaries[classvalue] = summarize(instances) #summarize is used to cal to mean and std
    return summaries

In [35]:
def calculateprobability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [36]:
def calculateclassprobabilities(summaries, inputvector):
    probabilities = {} # probabilities contains the all prob of all class of test data
    for classvalue, classsummaries in summaries.items():#class and attribute information as mean and sd
        probabilities[classvalue] = 1
        for i in range(len(classsummaries)):
            mean, stdev = classsummaries[i] #take mean and sd of every attribute for class 0 and 1 seperaely
            x = inputvector[i] #testvector's first attribute
            probabilities[classvalue] *= calculateprobability(x, mean, stdev);#use normal dist
    return probabilities

In [37]:
def predict(summaries, inputvector): #training and test data is passed
    probabilities = calculateclassprobabilities(summaries, inputvector)
    bestLabel, bestProb = None, -1
    for classvalue, probability in probabilities.items():#assigns that class which has he highest prob
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classvalue
    return bestLabel

PREDICTIONS

In [38]:
def getpredictions(summaries, testset):
    predictions = []
    for i in range(len(testset)):
        result = predict(summaries, testset[i])
        predictions.append(result)
    return predictions

ACCURACY

In [39]:
def getaccuracy(testset, predictions):
    correct = 0
    for i in range(len(testset)):
        if testset[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testset))) * 100.0

In [40]:
def main():
    filename = '5-dataset.csv'
    splitratio = 0.67
    dataset = loadcsv(filename);
     
    trainingset, testset = splitdataset(dataset, splitratio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingset), len(testset)))
    # prepare model
    summaries = summarizebyclass(trainingset);    
    #print(summaries)
    # test model
    predictions = getpredictions(summaries, testset) #find the predictions of test data with the training data
    accuracy = getaccuracy(testset, predictions)
    print('Accuracy of the classifier is : {0}%'.format(accuracy))

In [41]:
main()

Split 767 rows into train=513 and test=254 rows
Accuracy of the classifier is : 73.22834645669292%
